In [ ]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini chat - Multi-turn

## Set environment

### Package Install
* [google-cloud-aiplatform](https://cloud.google.com/python/docs/reference/aiplatform/latest)

In [1]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 32.9 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


### Authentication to access to GCP
* Only for Colab in Google Drive
* No need to do this process if in Colab Enteprise on Vertex AI.
* Refer to the [authentication ways](https://cloud.google.com/docs/authentication?hl=ko) in GCP

In [2]:
# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Lab Execution

### Define constants

In [3]:
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
MODEL_NAME = "gemini-1.5-flash" # @param {type:"string"}

### Import libraries

In [17]:
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    GenerationResponse,
    Tool,
    Part,
    ChatSession
)

from vertexai.preview.generative_models import grounding


### Initalize Vertex AI

Use system instructions when using multi-turn to keep consistency of communication.


In [19]:
# https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization
vertexai.init(project=PROJECT_ID, location=LOCATION)

system_instructions = [
 "당신은 가전제품을 구매하도록 도와주는 어시스턴트 입니다.",
 "질문 내용에 상세하게 답해주세요. "
]

# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.generative_models.GenerativeModel
model = GenerativeModel(MODEL_NAME,
                        system_instruction=system_instructions)

### Helper function

In [20]:
def interactive_chat(chat: ChatSession, question: str) -> str:
    """
    Generate a response from the model.

    Args:
      chat: ChatSession
        query string to generate response
      question: str
        whether to stream the response

    Returns:
      str: The generated response.

    """

    # Set model parameter : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
    }

    # Google Search grounding.
    tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

    # Configure satey setting : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes
    # Refer to the link to remove : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes#how_to_remove_automated_response_blocking_for_select_safety_attributes
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }

    text_response = []

    # https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.language_models.ChatSession
    responses = chat.send_message(
        question,
        generation_config = generation_config,
        safety_settings = safety_settings,
        tools = [tool],
        stream=False
    )

    return responses.text

#------------------------------------------------------------------------------------------------------------------------

def get_chat_history(chat):

  """
  Generate a response from the model.

  Args:
    chat: ChatSession
      query string to generate response

  Returns:
    None

  """

  history = " ".join([content.text for content in chat.history])

  print(f"---------------- Chat history ----------------\n{history} ")



### Execution

In [22]:
chat = model.start_chat()

while True:
  query = input('사용자: ')

  if query == '종료': break

  # chat_history = get_chat_history(chat)

  prompt = f"""
  당신은 가전제품을 구매하도록 도와 주는 AI Assistant 입니다.
  아래 질문에 대해서 친절하게 10줄 이내로 답해주세요.
  당신은 사용자가 적합한 제품을 살수 있도록 도와 줘야 합니다.

  질문 : {query}
  """

  response = interactive_chat(chat, prompt)
  display(Markdown(f"AI Agent : {response}"))
  print(f"------------------------------------ ")


사용자: 삼성 최신 TV 는 어떤게 있나요 ? 출시일도 알려주세요.


AI Agent : 삼성의 최신 TV는 2024년형 Neo QLED와 삼성 OLED TV입니다. 2024년 3월 15일에 국내 시장에 출시되었으며, AI 기능을 탑재하여 사용자 편의성을 높였습니다. 특히 Neo QLED 8K는 3세대 AI 8K 프로세서를 탑재하여 역대 삼성 TV 중 가장 뛰어난 성능을 자랑합니다. 2024년형 삼성 TV는 19년 연속 글로벌 1위를 목표로 하고 있으며, 프리미엄 TV와 초대형화 트렌드를 이끌고 있습니다. 


------------------------------------ 
사용자: LG 제품도 설명해주세요.


AI Agent : LG전자는 TV, 냉장고, 세탁기, 에어컨 등 다양한 가전제품을 생산하는 글로벌 기업입니다. 특히 OLED TV는 뛰어난 화질과 디자인으로 프리미엄 시장에서 인정받고 있습니다. 또한, LG ThinQ라는 AI 플랫폼을 통해 스마트홈 생태계를 구축하고 있으며, 사용자 편의성을 높이는 제품 개발에 힘쓰고 있습니다. 최근에는 롤러블 TV, 투명 OLED 등 혁신적인 기술을 선보이며 미래 가전 시장을 선도하고 있습니다.


------------------------------------ 
사용자: 과천에서 삼성제품을 어디서 살수 있나요 ?


AI Agent : 과천에서 삼성 제품을 구매하시려면, 삼성전자 서비스센터나 삼성스토어를 방문하시면 됩니다. 삼성전자 서비스센터는 제품 수리 및 A/S를 전문으로 하며, 삼성스토어는 다양한 삼성 제품을 전시하고 판매하는 곳입니다.  번개장터에서 삼성 990 PRO 4테라 중고 제품을 판매하는 사람이 과천역 근처에서 직거래 가능하다고 합니다.  삼성전자 서비스센터의 경우 과천에는 없고, 서울 강남구 논현 SVC점이 가까운 곳에 있습니다. 


------------------------------------ 
사용자: 정확한 주소를 알려주세요.


AI Agent :  삼성 제품을 구매할 수 있는 과천 지역 삼성 서비스센터는 과천에는 없고 서울 강남구 논현 SVC점이 가까운 곳에 있습니다.  정확한 주소는 삼성전자서비스 홈페이지에서 확인하실 수 있습니다.  삼성 제품을 구매할 수 있는 삼성 스토어는  서울 강남구 강남대로 452  에 위치해 있습니다.  


------------------------------------ 
사용자: 종료
